## Loading and Cleaning  Data

In [3]:
# -*- coding: utf-8 -*-

from __future__ import division, unicode_literals

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import dateutil
import datetime
import time

import re

import pickle

from genderizer.genderizer import Genderizer

# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

In [14]:
def load_pickles(start, stop):
    df = pd.DataFrame([])
    for i in range(stop, start-1, -1):
        df = df.append(pd.read_pickle('data/box_office_mojo_'+str(i)+'.pkl'), ignore_index=True)
    return df

## Load Pickle Point

In [15]:
# Load pickle data
box_office_mojo = load_pickles(1980, 2016) # 14269 records
omdb = pd.read_pickle('data/omdb.pkl') # 2000 records
bechdel = pd.read_pickle('data/bechdel.pkl') # 5420 records
polygraph = pd.read_pickle('data/polygraph.pkl') # 1944 records

In [16]:
# Initial drop of unnecessary/duplicate columns
bechdel.drop(['title', 'year'], axis=1, inplace=True)
polygraph.drop(['script_id', 'title', 'year', 'gross'], axis=1, inplace=True)
omdb.drop(['box_office', 'dvd', 'plot', 'poster', 'response', 'tomato_consensus', 'tomato_fresh', 
           'tomato_image', 'tomato_rotten', 'type', 'website'], axis=1, inplace=True)
box_office_mojo.drop('Director', axis=1, inplace=True)

In [17]:
# Merging dataframes by unique IMDB IDs
polygraph_bechdel = pd.merge(polygraph, bechdel, on = 'imdb_id') # 1244 records
omdb_polygraph_bechdel = pd.merge(omdb, polygraph_bechdel, on = 'imdb_id') # 1244 records

In [25]:
polygraph.describe()

gender,f,m,total_lines,ratio
count,1944.000000,1944.000000,1944.000000,1944.000000
mean,3019.236111,7515.173354,10534.409465,0.289650
std,2879.137982,5082.566016,6257.806830,0.192744
min,101.000000,109.000000,1175.000000,0.007170
25%,1146.250000,4159.000000,6425.000000,0.143259
50%,2125.500000,6450.500000,8958.000000,0.249238
75%,3893.750000,9459.750000,12771.250000,0.394902
max,25184.000000,57944.000000,67678.000000,0.993541


In [189]:
# Pickling for posterity
omdb_polygraph_bechdel.to_pickle('data/omdb_polygraph_bechdel.pkl')

In [190]:
# Including transformation functions
def to_date(datestring):
    try:
        date = dateutil.parser.parse(datestring)
        return date
    except:
        return datestring
    
def money_to_int(moneystring):
    num = [('million', 6), ('thousand', 3)]
    moneystring = moneystring.replace('$', '').replace(',', '').replace(' ','').replace('(Estimate)','').lower()
    for n in num:
        if n[0] in moneystring:
            moneystring = moneystring.replace(n[0], '')
            return int(moneystring) * np.power(10,n[1])
    else:
        return int(moneystring)

## Fuzzy Wuzzy Match - Not Used

In [191]:
# Using year for joins
# box_office_mojo['year'] = box_office_mojo['ReleaseDate'].dt.year
# omdb_polygraph_bechdel['year'] = omdb_polygraph_bechdel['year'].apply(float)
# merged = pd.merge(box_office_mojo, omdb_polygraph_bechdel, on = 'year', suffixes = ['_1', '_2'])

In [192]:
# merged['released'] = merged['released'].apply(to_date)
# merged['box-title-release'] = merged['Title'] + merged['ReleaseDate'].apply(str)
# merged['omdb-title-release'] = merged['title'] + merged['released'].apply(str)

In [193]:
# 1,224 - 1,117 records lost to exact merge
# 574,038 when merging on year

In [194]:
# def fuzztitle(row):
#     try:
#         return fuzz.ratio(row['box-title-release'], row['omdb-title-release'])
#     except:
#         return 0
    
# merged.loc[:, 'ratio'] = merged.apply(fuzztitle, axis = 1)
# merged[(merged['ratio']>80) & (merged['ratio']<100)].loc[:,['Title','title','ReleaseDate','released','ratio']]

## Final merge

In [195]:
box_office_mojo = box_office_mojo.rename(columns={'Title':'title'})
merged = pd.merge(box_office_mojo, omdb_polygraph_bechdel, on = 'title') # 1117 records

In [196]:
merged.head()

,title,DomesticTotalGross,Distributor,Runtime,MPAARating,ReleaseDate,Genre,ProductionBudget,actors,awards,...,tomato_user_meter,tomato_user_rating,tomato_user_reviews,writer,year,f,m,total_lines,ratio,bechdel
0,Star Wars: The Force Awakens,936662225.0,Buena Vista,136,PG-13,2015-12-18 00:00:00,Sci-Fi Fantasy,2.45e+08,"Harrison Ford, Mark Hamill, Carrie Fisher, Ada...",Nominated for 5 Oscars. Another 48 wins & 104 ...,...,89,4.3,219127,"Lawrence Kasdan, J.J. Abrams, Michael Arndt, G...",2015,1856.0,4810.0,6666.0,0.278428,1.0
1,Inside Out,356461711.0,Buena Vista,94,PG,2015-06-19 00:00:00,Animation,1.75e+08,"Amy Poehler, Phyllis Smith, Richard Kind, Bill...",Won 1 Oscar. Another 91 wins & 95 nominations.,...,89,4.3,130887,"Pete Docter (original story by), Ronnie Del Ca...",2015,4836.0,2696.0,7532.0,0.642061,1.0
2,Minions,336045770.0,Universal,91,PG,2015-07-10 00:00:00,Animation,7.4e+07,"Sandra Bullock, Jon Hamm, Michael Keaton, Alli...",Nominated for 1 BAFTA Film Award. Another 18 n...,...,49,3.3,135378,Brian Lynch,2015,1828.0,1800.0,3628.0,0.503859,1.0
3,The Martian,228433663.0,Fox,141,PG-13,2015-10-02 00:00:00,Sci-Fi,1.08e+08,"Matt Damon, Jessica Chastain, Kristen Wiig, Je...",Nominated for 7 Oscars. Another 35 wins & 171 ...,...,91,4.3,127183,"Drew Goddard (screenplay), Andy Weir (book)",2015,2864.0,7353.0,10217.0,0.280317,1.0
4,Straight Outta Compton,161197785.0,Universal,147,R,2015-08-14 00:00:00,Drama,2.8e+07,"O'Shea Jackson Jr., Corey Hawkins, Jason Mitch...",Nominated for 1 Oscar. Another 24 wins & 39 no...,...,92,4.3,73738,"Jonathan Herman (screenplay), Andrea Berloff (...",2015,675.0,9855.0,10530.0,0.064103,0.0


In [197]:
# Second drop of unnecessary/duplicate columns and renaming of columns

# Duplicates: Genres (Multiple), Runtime, MPAA Rating, Release Date
# Unnecessary (for now): Distributor, Producer

# Drop - Genre, runtime, rated, released, Distributor, production

merged.drop(['Genre', 'runtime', 'rated', 'released', 'Distributor', 'production'], axis=1, inplace=True)
merged = merged.rename(columns = {'DomesticTotalGross': 'domestic_total_gross', 'Runtime': 'runtime', 'MPAARating': 'mpaa', 
                        'ReleaseDate': 'release_date', 'ProductionBudget': 'production_budget'})

In [198]:
# Pickle and retrieve dataframe
merged.to_pickle('data/merged.pkl')

## Load Pickle Point

In [4]:
merged = pd.read_pickle('data/merged.pkl')

## Mutating and Dummies

In [5]:
# Helper function
def find_chosen_one(i, s):
    return int(i in s)

In [6]:
# Create # wins and # nominations
def return_wins(s):
    wins_regex = re.compile('((?<=[Ww]on )[0-9]|[0-9]*(?= [Ww]ins*))')
    wins = 0
    for i in re.findall(wins_regex, s):
        try:
            wins += int(i)
        except:
            pass
    return wins

def return_nominations(s):
    nominations_regex = re.compile('((?<=[Nn]ominated for )[0-9]|[0-9]*(?= [Nn]ominations*))')
    nominations = 0
    for i in re.findall(nominations_regex, s):
        try:
            nominations += int(i)
        except:
            pass
    return nominations

def create_awards(df):
    df['wins'] = df['awards'].apply(return_wins)
    df['nominations'] = df['awards'].apply(return_nominations)
    return df

## Getting Genderizations

In [7]:
# Returns the gender of a name based on the first name (defined as first word in name)
def return_gender(name):
    return Genderizer.detect(firstName = name.split(' ')[0])

# Returns the ratio of female to male agents (0 if all male, 1 if all female, fractional if mix)
# If gender is unknown, assume male
def genderize(s):
    l = []
    for i in s.split(', '):
        l.append(return_gender(i))
    return l.count('female')/len(l)

# Creates a genderization column in a df
def create_genderization(df, column):
    colname = column + '_genderization'
    df[colname] = df[column].apply(genderize)
    return df

## Getting Binaries

In [8]:
# Creates a dummy variable for whether the 'chosen one' (e.g., "USA", "English") is present or not
# Used for identifying foreign/non-English language films
def create_binaries(df, column, chosen_one):
    colname = column + '_' + chosen_one.lower()
    df[colname] = df[column].apply(lambda x: find_chosen_one(chosen_one, x)) 
    return df

## Getting Dummies

In [9]:
# Turn list-like, categorical columns into dummy variables
# Getting list of unique categorical levels
def find_unique(df, column):
    l = [i for i in df.loc[:,column]]
    l = reduce(lambda x, y: x + ', ' + y, l).split(',')
    return list(set(map(lambda x: x.strip(), l)))

# Creating dummy variables from each level (for multiple categories in single cell)
def create_dummies(df, column):
    l = find_unique(df, column)
    for i in l:
        colname = column + '_' + i.replace(' ','_').lower()
        df[colname] = df[column].apply(lambda x: find_chosen_one(i, x))
    return df

In [11]:
# Ready to edit dataframe for variables!
final = merged.copy()

# Create Awards - 'wins', 'nominations', drop awards
final = create_awards(final)
final.drop(['awards'], axis=1, inplace=True)

# Person Categories - Genderize - 'writer', 'actors', 'director', drop originals and also f, m
final = create_genderization(final, 'writer')
final = create_genderization(final, 'actors')
final = create_genderization(final, 'director')
final.drop(['writer', 'actors', 'director', 'f', 'm'], axis=1, inplace=True)

# Binary Categories - Create Binaries - 'country_usa', 'language_english', drop originals
final = create_binaries(final, 'country', 'USA')
final = create_binaries(final, 'language', 'English')
final.drop(['country','language'], axis=1, inplace=True)

# Unique Categories - Create Dummies - 'genre_...', 'mpaa_...' - possibly include producer & distributor in here
final = create_dummies(final, 'genre')
final = pd.get_dummies(final, columns = ['mpaa'])
final.drop(['genre'], axis=1, inplace=True)

In [206]:
# Final cleanup
final[final.duplicated(subset = 'imdb_id', keep=False)].loc[:,['imdb_id', 'title','release_date', 'year']]

,imdb_id,title,release_date,year
27,tt3170832,Room,2015-10-16 00:00:00,2015
28,tt3170832,Room,2006-04-06 00:00:00,2015
47,tt3569230,Legend,2015-11-20 00:00:00,2015
48,tt0089469,Legend,2015-11-20 00:00:00,1985
49,tt3569230,Legend,1986-04-18 00:00:00,2015
50,tt0089469,Legend,1986-04-18 00:00:00,1985
275,tt0087538,The Karate Kid,2010-06-11 00:00:00,1984
276,tt0087538,The Karate Kid,1984-06-22 00:00:00,1984
591,tt0104412,Hero,2004-08-27 00:00:00,1992
592,tt0104412,Hero,1992-10-02 00:00:00,1992


In [207]:
# Removing duplicates manually (just a handful)
final.drop([28, 48, 49, 275, 591, 1082], inplace=True)

In [208]:
# Changing datatypes
final['release_date'] = final['release_date'].apply(to_date).apply(datetime.date.toordinal)
final['imdb_votes'] = final['imdb_votes'].apply(money_to_int)

def change_type(df):
    objects = df.select_dtypes(include=['object']).drop(['title','imdb_id'], axis = 1).columns
    for i in objects:
        try:
            df[i] = df[i].replace('N/A',np.nan)
            df[i] = pd.to_numeric(df[i], errors = 'raise')
        except:
            print i
    return df

# Check dtype
# for i in final3.columns:
#     print i, final3[i].dtype

In [210]:
final2 = change_type(final)

In [234]:
# dropping movies with no production budget listed
final3 = final2[final2['production_budget'].notnull()].copy()

In [235]:
# Replacing NA values with mean
nas = final3.isnull().sum()
nas_index = nas[nas > 0].index

In [236]:
def replace_nas(df, columns):
    for i in df[columns]:
        final[i] = final[i].fillna(value = final[i].mean())
    return df

In [262]:
final4 = replace_nas(final3, nas_index)

In [263]:
# Identifying outliers
final4['gross_to_budget'] = final4['domestic_total_gross'] / final4['production_budget']
final4.sort_values(by = 'gross_to_budget', ascending = False)

,title,domestic_total_gross,runtime,release_date,production_budget,imdb_id,imdb_rating,imdb_votes,metascore,tomato_meter,...,genre_musical,genre_biography,genre_history,mpaa_G,mpaa_NC-17,mpaa_PG,mpaa_PG-13,mpaa_R,mpaa_Unrated,gross_to_budget
744,The Blair Witch Project,140539099.0,86.000000,729951,60000.0,tt0185937,6.4,188041,81.000000,86.000000,...,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,2342.318317
911,Clerks,3151130.0,92.000000,728220,27000.0,tt0109445,7.8,183008,70.000000,88.000000,...,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,116.708519
637,My Big Fat Greek Wedding,241438208.0,95.000000,730959,5000000.0,tt0259446,6.6,102640,62.000000,76.000000,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,48.287642
865,Swingers,4555020.0,96.000000,728950,200000.0,tt0117802,7.4,64404,71.000000,87.000000,...,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,22.775100
174,Sinister,48086903.0,110.000000,734788,3000000.0,tt1922777,6.8,156939,53.000000,63.000000,...,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,16.028968
976,Home Alone,285761243.0,105.000000,726787,18000000.0,tt0099785,7.5,313028,63.000000,55.000000,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,15.875625
1083,Terms of Endearment,108423489.0,131.000000,724237,8000000.0,tt0086425,7.4,40327,79.000000,88.000000,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,13.552936
900,Pulp Fiction,107928762.0,154.000000,728215,8000000.0,tt0110912,8.9,1334945,94.000000,94.000000,...,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,13.491095
978,Pretty Woman,178406268.0,119.000000,726549,14000000.0,tt0100405,6.9,214849,51.000000,62.000000,...,0,0,0,0.0,0.0,0.0,0.0,1.0,0.0,12.743305
1099,Raiders of the Lost Ark,212222025.0,115.000000,723343,18000000.0,tt0082971,8.5,665774,85.000000,94.000000,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,11.790112


In [267]:
# Excluding movies where gross to budget ratio is high (skewed)
final5 = final4[final4['gross_to_budget']<10].copy()
final5.drop('gross_to_budget', axis=1, inplace=True)

In [270]:
movies = final5.set_index(['title', 'imdb_id'])

In [271]:
movies.to_pickle('data/movies.pkl')

## Load Pickle Point

In [4]:
movies = pd.read_pickle('data/movies.pkl')

In [30]:
bechdel.describe()

,bechdel
count,5420.000000
mean,0.614207
std,0.486827
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [31]:
omdb_polygraph_bechdel.describe()

,f,m,total_lines,ratio,bechdel
count,1224.000000,1224.000000,1224.000000,1224.000000,1224.000000
mean,3279.626634,7480.081699,10759.708333,0.307899,0.526144
std,3137.463790,5113.104475,6314.384581,0.202198,0.499520
min,102.000000,114.000000,1461.000000,0.007170,0.000000
25%,1237.000000,4134.000000,6678.750000,0.150427,0.000000
50%,2275.000000,6463.000000,9096.000000,0.257162,1.000000
75%,4181.000000,9477.500000,13072.250000,0.431512,1.000000
max,25184.000000,57944.000000,67678.000000,0.993541,1.000000


In [34]:
((omdb_polygraph_bechdel['f']) / (omdb_polygraph_bechdel['total_lines'])).median()

0.25716207421643156